In [1]:
import pandas as pd
import numpy as np

In [2]:
from Demographics_ETL import extract_data, transform_data, load_data

In [3]:
%%HTML
<style type="text/css">

table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}

</style>

In [4]:
INDEX_NAME='demographics_v2_test'
TYPE_NAME ='_doc' 
uuid = 5100
from elasticsearch import Elasticsearch
#elastic = Elasticsearch(HOST="localhost",PORT=9200)

# elastic = Elasticsearch([{'host':'142.93.213.146','port':9200}])

In [5]:
f_2012=pd.read_csv("combined_2011_12_district.csv")
f_2013=pd.read_csv("combined_2012_13_district.csv")
f_2014=pd.read_csv("combined_2013_14_district.csv")
f_2015=pd.read_csv("combined_2014_15_district.csv")
f_2016=pd.read_csv("combined_2015_16_district.csv")
f_2017=pd.read_csv("combined_2016_17_district.csv")
f_2018=pd.read_csv("combined_2017_18_district.csv")
f_2019=pd.read_csv("combined_2018_19_district.csv")
f_2020=pd.read_csv("combined_2019_20_district.csv")
f_2021=pd.read_csv("combined_2020_21_district.csv")

f_2012.columns=f_2012.columns.str.replace("<br>"," ")
f_2012.columns=f_2012.columns.str.replace("/"," ")
f_2013.columns=f_2012.columns.str.replace("<br>"," ")
f_2013.columns=f_2012.columns.str.replace("/"," ")
f_2014.columns=f_2012.columns.str.replace("<br>"," ")
f_2014.columns=f_2012.columns.str.replace("/"," ")
f_2015.columns=f_2012.columns.str.replace("<br>"," ")
f_2015.columns=f_2012.columns.str.replace("/"," ")
f_2016.columns=f_2012.columns.str.replace("<br>"," ")
f_2016.columns=f_2012.columns.str.replace("/"," ")
f_2017.columns=f_2012.columns.str.replace("<br>"," ")
f_2017.columns=f_2012.columns.str.replace("/"," ")
f_2018.columns=f_2012.columns.str.replace("<br>"," ")
f_2018.columns=f_2012.columns.str.replace("/"," ")
f_2019.columns=f_2012.columns.str.replace("<br>"," ")
f_2019.columns=f_2012.columns.str.replace("/"," ")
f_2020.columns=f_2012.columns.str.replace("<br>"," ")
f_2020.columns=f_2012.columns.str.replace("/"," ")
f_2021.columns=f_2012.columns.str.replace("<br>"," ")
f_2021.columns=f_2012.columns.str.replace("/"," ")

In [6]:
li=[f_2012,f_2013,f_2014,f_2015,f_2016,f_2017,f_2018,f_2019,f_2020,f_2021]
yr=["2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]

In [7]:
col=list(f_2012.columns)
col=col[4:]

In [8]:
for j in col:
    final=pd.DataFrame()
    for i in range(1,11):
        n=li[i-1]
        n.fillna("", inplace = True) 
        if(i==1):

            n=n[["State Name","District Name",j]]
            n.rename(columns={j:yr[i-1]},inplace=True)
            final=n
        else:
            #n.drop(columns={"Unnamed: 0","Sno."},inplace=True)
            n=n[["State Name","District Name",j]]
            n.rename(columns={j:yr[i-1]},inplace=True)
            n.drop(columns={'State Name','District Name'},inplace=True)
            final=pd.concat([final,n],axis=1,sort=False)
    
    final.loc[637,"State Name"]="Total"
    final.loc[637,"District Name"]=0  
    final=final.replace(np.nan,"",regex=True)
    
    PATH = 'MNREGA/'+j+'/'
    states_list=final["State Name"].unique()
    states_list=states_list.tolist()
    statedic = {}
    for i in range(len(states_list)):
        j = states_list[i]
        df = pd.DataFrame(final.loc[final['State Name'] == j ])
        df = df.drop(columns=df.columns[0])
        df = df.rename(columns={df.columns[0]:"Year"})
        df.set_index('Year',inplace=True)
        df = df.T
        df = df.loc[:,~df.columns.duplicated()]
        df = df.replace(np.nan, '', regex=True)
        df.columns = df.columns.astype(str)
        df.columns = df.columns.str.title()
        statedic[states_list[i]]=df
    new_list=states_list.copy()
    new_list[0]="Andaman"
    new_list[1]="Dadra_and_Nagar_Haveli"
    a=[]
    for i in new_list:
    
        i=str(i).title()
        i=str(i).replace(" ","_")
        a.append(i)
    a[33]="All Districts"
    a1=a.copy()
    for i in range(0,34):
        globals()[a[i]]=statedic[states_list[i]]
        states_list[i]= states_list[i].title()
        doc_source={
            "Doc_Source":"MNREGA Portal",
            "Path":PATH,
            "Units":"Number of People",
            "Mode":"District",
            "State":states_list[i]
            }
       # globals()[a1[i]+'_meta']= transform_data(a[i],doc_source)
        globals()[a1[i]+'_meta']=transform_data(globals()[a[i]],doc_source)
        uuid = load_data(globals()[a[i]],globals()[a1[i]+'_meta'],INDEX_NAME,TYPE_NAME,uuid )
        
    
    

C:\Users\Abhishek\Anaconda3\lib\site-packages\pandas\core\frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Abhishek\Anaconda3\lib\site-packages\pandas\core\frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


uuid is 5100
uuid is 5101
uuid is 5102
uuid is 5103
uuid is 5104
uuid is 5105
uuid is 5106
uuid is 5107
uuid is 5108
uuid is 5109
uuid is 5110
uuid is 5111
uuid is 5112
uuid is 5113
uuid is 5114
uuid is 5115
uuid is 5116
uuid is 5117
uuid is 5118
uuid is 5119
uuid is 5120
uuid is 5121
uuid is 5122
uuid is 5123
uuid is 5124
uuid is 5125
uuid is 5126
uuid is 5127
uuid is 5128
uuid is 5129
uuid is 5130
uuid is 5131
uuid is 5132
uuid is 5133
uuid is 5134
uuid is 5135
uuid is 5136
uuid is 5137
uuid is 5138
uuid is 5139
uuid is 5140
uuid is 5141
uuid is 5142
uuid is 5143
uuid is 5144
uuid is 5145
uuid is 5146
uuid is 5147
uuid is 5148
uuid is 5149
uuid is 5150
uuid is 5151
uuid is 5152
uuid is 5153
uuid is 5154
uuid is 5155
uuid is 5156
uuid is 5157
uuid is 5158
uuid is 5159
uuid is 5160
uuid is 5161
uuid is 5162
uuid is 5163
uuid is 5164
uuid is 5165
uuid is 5166
uuid is 5167
uuid is 5168
uuid is 5169
uuid is 5170
uuid is 5171
uuid is 5172
uuid is 5173
uuid is 5174
uuid is 5175
uuid is 5176

uuid is 5731
uuid is 5732
uuid is 5733
uuid is 5734
uuid is 5735
uuid is 5736
uuid is 5737
uuid is 5738
uuid is 5739
uuid is 5740
uuid is 5741
uuid is 5742
uuid is 5743
uuid is 5744
uuid is 5745
uuid is 5746
uuid is 5747
uuid is 5748
uuid is 5749
uuid is 5750
uuid is 5751
uuid is 5752
uuid is 5753
uuid is 5754
uuid is 5755
uuid is 5756
uuid is 5757
uuid is 5758
uuid is 5759
uuid is 5760
uuid is 5761
uuid is 5762
uuid is 5763
uuid is 5764
uuid is 5765
uuid is 5766
uuid is 5767
uuid is 5768
uuid is 5769
uuid is 5770
uuid is 5771
uuid is 5772
uuid is 5773
uuid is 5774
uuid is 5775
uuid is 5776
uuid is 5777
uuid is 5778
uuid is 5779
uuid is 5780
uuid is 5781
uuid is 5782
uuid is 5783
uuid is 5784
uuid is 5785
uuid is 5786
uuid is 5787
uuid is 5788
uuid is 5789
uuid is 5790
uuid is 5791
uuid is 5792
uuid is 5793
uuid is 5794
uuid is 5795
uuid is 5796
uuid is 5797
uuid is 5798
uuid is 5799
uuid is 5800
uuid is 5801
uuid is 5802
uuid is 5803
uuid is 5804
uuid is 5805
uuid is 5806
uuid is 5807

uuid is 6363
uuid is 6364
uuid is 6365
uuid is 6366
uuid is 6367
uuid is 6368
uuid is 6369
uuid is 6370
uuid is 6371
uuid is 6372
uuid is 6373
uuid is 6374
uuid is 6375
uuid is 6376
uuid is 6377
uuid is 6378
uuid is 6379
uuid is 6380
uuid is 6381
uuid is 6382
uuid is 6383
uuid is 6384
uuid is 6385
uuid is 6386
uuid is 6387
uuid is 6388
uuid is 6389
uuid is 6390
uuid is 6391
uuid is 6392
uuid is 6393
uuid is 6394
uuid is 6395
uuid is 6396
uuid is 6397
uuid is 6398
uuid is 6399
uuid is 6400
uuid is 6401
uuid is 6402
uuid is 6403
uuid is 6404
uuid is 6405
uuid is 6406
uuid is 6407
uuid is 6408
uuid is 6409
uuid is 6410
uuid is 6411
uuid is 6412
uuid is 6413
uuid is 6414
uuid is 6415
uuid is 6416
uuid is 6417
uuid is 6418
uuid is 6419
uuid is 6420
uuid is 6421
uuid is 6422
uuid is 6423
uuid is 6424
uuid is 6425
uuid is 6426
uuid is 6427
uuid is 6428
uuid is 6429
uuid is 6430
uuid is 6431
uuid is 6432
uuid is 6433
uuid is 6434
uuid is 6435
uuid is 6436
uuid is 6437
uuid is 6438
uuid is 6439

uuid is 6994
uuid is 6995
uuid is 6996
uuid is 6997
uuid is 6998
uuid is 6999
uuid is 7000
uuid is 7001
uuid is 7002
uuid is 7003
uuid is 7004
uuid is 7005
uuid is 7006
uuid is 7007
uuid is 7008
uuid is 7009
uuid is 7010
uuid is 7011
uuid is 7012
uuid is 7013
uuid is 7014
uuid is 7015
uuid is 7016
uuid is 7017
uuid is 7018
uuid is 7019
uuid is 7020
uuid is 7021
uuid is 7022
uuid is 7023
uuid is 7024
uuid is 7025
uuid is 7026
uuid is 7027
uuid is 7028
uuid is 7029
uuid is 7030
uuid is 7031
uuid is 7032
uuid is 7033
uuid is 7034
uuid is 7035
uuid is 7036
uuid is 7037
uuid is 7038
uuid is 7039
uuid is 7040
uuid is 7041
uuid is 7042
uuid is 7043
uuid is 7044
uuid is 7045
uuid is 7046
uuid is 7047
uuid is 7048
uuid is 7049
uuid is 7050
uuid is 7051
uuid is 7052
uuid is 7053
uuid is 7054
uuid is 7055
uuid is 7056
uuid is 7057
uuid is 7058
uuid is 7059
uuid is 7060
uuid is 7061
uuid is 7062
uuid is 7063
uuid is 7064
uuid is 7065
uuid is 7066
uuid is 7067
uuid is 7068
uuid is 7069
uuid is 7070

...

uuid is 68781
uuid is 68782
uuid is 68783
uuid is 68784
uuid is 68785
uuid is 68786
uuid is 68787
uuid is 68788
uuid is 68789
uuid is 68790
uuid is 68791
uuid is 68792
uuid is 68793
uuid is 68794
uuid is 68795
uuid is 68796
uuid is 68797
uuid is 68798
uuid is 68799
uuid is 68800
uuid is 68801
uuid is 68802
uuid is 68803
uuid is 68804
uuid is 68805
uuid is 68806
uuid is 68807
uuid is 68808
uuid is 68809
uuid is 68810
uuid is 68811
uuid is 68812
uuid is 68813
uuid is 68814
uuid is 68815
uuid is 68816
uuid is 68817
uuid is 68818
uuid is 68819
uuid is 68820
uuid is 68821
uuid is 68822
uuid is 68823
uuid is 68824
uuid is 68825
uuid is 68826
uuid is 68827
uuid is 68828
uuid is 68829
uuid is 68830
uuid is 68831
uuid is 68832
uuid is 68833
uuid is 68834
uuid is 68835
uuid is 68836
uuid is 68837
uuid is 68838
uuid is 68839
uuid is 68840
uuid is 68841
uuid is 68842
uuid is 68843
uuid is 68844
uuid is 68845
uuid is 68846
uuid is 68847
uuid is 68848
uuid is 68849
uuid is 68850
uuid is 68851
uuid i

uuid is 69367
uuid is 69368
uuid is 69369
uuid is 69370
uuid is 69371
uuid is 69372
uuid is 69373
uuid is 69374
uuid is 69375
uuid is 69376
uuid is 69377
uuid is 69378
uuid is 69379
uuid is 69380
uuid is 69381
uuid is 69382
uuid is 69383
uuid is 69384
uuid is 69385
uuid is 69386
uuid is 69387
uuid is 69388
uuid is 69389
uuid is 69390
uuid is 69391
uuid is 69392
uuid is 69393
uuid is 69394
uuid is 69395
uuid is 69396
uuid is 69397
uuid is 69398
uuid is 69399
uuid is 69400
uuid is 69401
uuid is 69402
uuid is 69403
uuid is 69404
uuid is 69405
uuid is 69406
uuid is 69407
uuid is 69408
uuid is 69409
uuid is 69410
uuid is 69411
uuid is 69412
uuid is 69413
uuid is 69414
uuid is 69415
uuid is 69416
uuid is 69417
uuid is 69418
uuid is 69419
uuid is 69420
uuid is 69421
uuid is 69422
uuid is 69423
uuid is 69424
uuid is 69425
uuid is 69426
uuid is 69427
uuid is 69428
uuid is 69429
uuid is 69430
uuid is 69431
uuid is 69432
uuid is 69433
uuid is 69434
uuid is 69435
uuid is 69436
uuid is 69437
uuid i

uuid is 69953
uuid is 69954
uuid is 69955
uuid is 69956
uuid is 69957
uuid is 69958
uuid is 69959
uuid is 69960
uuid is 69961
uuid is 69962
uuid is 69963
uuid is 69964
uuid is 69965
uuid is 69966
uuid is 69967
uuid is 69968
uuid is 69969
uuid is 69970
uuid is 69971
uuid is 69972
uuid is 69973
uuid is 69974
uuid is 69975
uuid is 69976
uuid is 69977
uuid is 69978
uuid is 69979
uuid is 69980
uuid is 69981
uuid is 69982
uuid is 69983
uuid is 69984
uuid is 69985
uuid is 69986
uuid is 69987
uuid is 69988
uuid is 69989
uuid is 69990
uuid is 69991
uuid is 69992
uuid is 69993
uuid is 69994
uuid is 69995
uuid is 69996
uuid is 69997
uuid is 69998
uuid is 69999
uuid is 70000
uuid is 70001
uuid is 70002
uuid is 70003
uuid is 70004
uuid is 70005
uuid is 70006
uuid is 70007
uuid is 70008
uuid is 70009
uuid is 70010
uuid is 70011
uuid is 70012
uuid is 70013
uuid is 70014
uuid is 70015
uuid is 70016
uuid is 70017
uuid is 70018
uuid is 70019
uuid is 70020
uuid is 70021
uuid is 70022
uuid is 70023
uuid i

uuid is 70540
uuid is 70541
uuid is 70542
uuid is 70543
uuid is 70544
uuid is 70545
uuid is 70546
uuid is 70547
uuid is 70548
uuid is 70549
uuid is 70550
uuid is 70551
uuid is 70552
uuid is 70553
uuid is 70554
uuid is 70555
uuid is 70556
uuid is 70557
uuid is 70558
uuid is 70559
uuid is 70560
uuid is 70561
uuid is 70562
uuid is 70563
uuid is 70564
uuid is 70565
uuid is 70566
uuid is 70567
uuid is 70568
uuid is 70569
uuid is 70570
uuid is 70571
uuid is 70572
uuid is 70573
uuid is 70574
uuid is 70575
uuid is 70576
uuid is 70577
uuid is 70578
uuid is 70579
uuid is 70580
uuid is 70581
uuid is 70582
uuid is 70583
uuid is 70584
uuid is 70585
uuid is 70586
uuid is 70587
uuid is 70588
uuid is 70589
uuid is 70590
uuid is 70591
uuid is 70592
uuid is 70593
uuid is 70594
uuid is 70595
uuid is 70596
uuid is 70597
uuid is 70598
uuid is 70599
uuid is 70600
uuid is 70601
uuid is 70602
uuid is 70603
uuid is 70604
uuid is 70605
uuid is 70606
uuid is 70607
uuid is 70608
uuid is 70609
uuid is 70610
uuid i

uuid is 71126
uuid is 71127
uuid is 71128
uuid is 71129
uuid is 71130
uuid is 71131
uuid is 71132
uuid is 71133
uuid is 71134
uuid is 71135
uuid is 71136
uuid is 71137
uuid is 71138
uuid is 71139
uuid is 71140
uuid is 71141
uuid is 71142
uuid is 71143
uuid is 71144
uuid is 71145
uuid is 71146
uuid is 71147
uuid is 71148
uuid is 71149
uuid is 71150
uuid is 71151
uuid is 71152
uuid is 71153
uuid is 71154
uuid is 71155
uuid is 71156
uuid is 71157
uuid is 71158
uuid is 71159
uuid is 71160
uuid is 71161
uuid is 71162
uuid is 71163
uuid is 71164
uuid is 71165
uuid is 71166
uuid is 71167
uuid is 71168
uuid is 71169
uuid is 71170
uuid is 71171
uuid is 71172
uuid is 71173
uuid is 71174
uuid is 71175
uuid is 71176
uuid is 71177
uuid is 71178
uuid is 71179
uuid is 71180
uuid is 71181
uuid is 71182
uuid is 71183
uuid is 71184
uuid is 71185
uuid is 71186
uuid is 71187
uuid is 71188
uuid is 71189
uuid is 71190
uuid is 71191
uuid is 71192
uuid is 71193
uuid is 71194
uuid is 71195
uuid is 71196
uuid i

In [139]:
a1

NameError: name 'a1' is not defined

In [93]:
states_list=final["State Name"].unique()
states_list=states_list.tolist()
states_list

['ANDAMAN AND NICOBAR',
 'DADRA & NAGAR HAVELI',
 'DAMAN & DIU',
 'LAKSHADWEEP',
 'NAGALAND',
 'ANDHRA PRADESH',
 'ARUNACHAL PRADESH',
 'ASSAM',
 'BIHAR',
 'CHHATTISGARH',
 'GOA',
 'GUJARAT',
 'HARYANA',
 'HIMACHAL PRADESH',
 'JAMMU AND KASHMIR',
 'JHARKHAND',
 'KARNATAKA',
 'KERALA',
 'MADHYA PRADESH',
 'MAHARASHTRA',
 'MANIPUR',
 'MEGHALAYA',
 'MIZORAM',
 'ODISHA',
 'PUDUCHERRY',
 'PUNJAB',
 'RAJASTHAN',
 'SIKKIM',
 'TAMIL NADU',
 'TRIPURA',
 'UTTAR PRADESH',
 'UTTARAKHAND',
 'WEST BENGAL',
 nan]

In [94]:
statedic = {}
for i in range(len(states_list)):
        j = states_list[i]
        df = pd.DataFrame(final.loc[final['State Name'] == j ])
        df = df.drop(columns=df.columns[0])
        df = df.rename(columns={df.columns[0]:"Year"})
        df.set_index('Year',inplace=True)
        df = df.T
        df = df.loc[:,~df.columns.duplicated()]
        df = df.replace(np.nan, '', regex=True)
        df.columns = df.columns.astype(str)
        df.columns = df.columns.str.title()
        statedic[states_list[i]]=df

In [95]:
statedic

{'ANDAMAN AND NICOBAR': Year  Nicobars  North And Middle Andaman  South Andaman
 2012    2882.0                   17687.0        24827.0
 2013    5149.0                   18292.0        25048.0
 2014    5512.0                   19765.0        25374.0
 2015    4245.0                   20511.0        25278.0
 2016    5737.0                   20967.0        25337.0
 2017    5620.0                   20234.0        10170.0
 2018    5585.0                   19848.0        10081.0
 2019       0.0                       0.0            0.0
 2020   74746.0                   23827.0        53981.0
 2021   74780.0                   23848.0        53789.0,
 'DADRA & NAGAR HAVELI': Year  Dadra And Nagar Haveli
 2012                  7845.0
 2013                  8176.0
 2014                  8157.0
 2015                  8157.0
 2016                  8157.0
 2017                  8157.0
 2018                  9141.0
 2019                     0.0
 2020                 21738.0
 2021                 217

In [96]:
new_list=states_list.copy()
new_list[0]="Andaman"
new_list[1]="Dadra_and_Nagar_Haveli"
a=[]
for i in new_list:
    
    i=str(i).title()
    i=str(i).replace(" ","_")
    a.append(i)
a[33]="All Districts"

In [150]:
for i in range(0,34):
    globals()[a[i]]=statedic[states_list[i]]

In [153]:
globals()[a[0]]

Year,Nicobars,North And Middle Andaman,South Andaman
2012,2882.0,17687.0,24827.0
2013,5149.0,18292.0,25048.0
2014,5512.0,19765.0,25374.0
2015,4245.0,20511.0,25278.0
2016,5737.0,20967.0,25337.0
2017,5620.0,20234.0,10170.0
2018,5585.0,19848.0,10081.0
2019,0.0,0.0,0.0
2020,74746.0,23827.0,53981.0
2021,74780.0,23848.0,53789.0


In [111]:
Kerala

Year,Alappuzha,Ernakulam,Idukki,Kannur,Kasargod,Kollam,Kottayam,Kozhikode,Malappuram,Palakkad,Pathanamthitta,Thiruvananthapuram,Thrissur,Wayanad
2012,174564.0,134969.0,126286.0,104670.0,71251.0,152583.0,122188.0,168978.0,165342.0,164459.0,77745.0,208040.0,123216.0,92483.0
2013,214764.0,168291.0,155486.0,161235.0,98141.0,233141.0,148297.0,214507.0,219490.0,225193.0,112535.0,277997.0,212210.0,112903.0
2014,229834.0,183182.0,165737.0,180430.0,112187.0,264276.0,160702.0,242020.0,248929.0,255549.0,120783.0,313027.0,236703.0,121697.0
2015,239323.0,198850.0,170868.0,190925.0,122132.0,283957.0,167148.0,259180.0,273046.0,280716.0,127580.0,331932.0,247998.0,134486.0
2016,248534.0,206937.0,177993.0,203880.0,130704.0,307515.0,172606.0,278709.0,294569.0,305806.0,132377.0,349506.0,259290.0,139604.0
2017,251982.0,208764.0,177588.0,195305.0,139313.0,311938.0,172739.0,274183.0,291748.0,316238.0,129684.0,364738.0,259312.0,130870.0
2018,259525.0,216645.0,184015.0,204154.0,145792.0,328969.0,179252.0,284268.0,304121.0,328549.0,134181.0,375247.0,267326.0,136186.0
2019,269131.0,229840.0,192489.0,213726.0,153047.0,349708.0,188491.0,297864.0,318479.0,347552.0,139355.0,390681.0,278595.0,143776.0
2020,309115.0,331375.0,359114.0,145035.0,402520.0,287098.0,146619.0,84979.0,125235.0,136872.0,85511.0,305197.0,155157.0,203444.0
2021,370655.0,151796.0,418429.0,303529.0,150427.0,87070.0,133750.0,145968.0,94517.0,303556.0,183006.0,213136.0,144978.0,52539.0


In [112]:
Karnataka

Year,Bagalkote,Ballari,Belagavi,Bengaluru,Bengaluru Rural,Bidar,Chamaraja Nagara,Chikkaballapura,Chikkamagaluru,Chitradurga,...,Mandya,Mysuru,Raichur,Ramanagara,Shivamogga,Tumakuru,Udupi,Uttara Kannada,Vijaypura,Yadgir
2012,200218.0,234294.0,541233.0,60995.0,77394.0,206324.0,103548.0,136856.0,131608.0,239867.0,...,204341.0,166307.0,234654.0,134087.0,183475.0,268194.0,35148.0,142841.0,352082.0,236283.0
2013,202567.0,234200.0,541100.0,51383.0,76537.0,211305.0,114417.0,138559.0,129897.0,247851.0,...,173559.0,177586.0,245480.0,136843.0,180950.0,283677.0,35793.0,142402.0,260081.0,196923.0
2014,208158.0,207401.0,499636.0,49802.0,80078.0,188989.0,129339.0,151482.0,128335.0,255846.0,...,188105.0,196150.0,258092.0,162617.0,177537.0,312869.0,40628.0,144072.0,248028.0,212927.0
2015,199560.0,215978.0,493664.0,49213.0,75516.0,170572.0,131404.0,158980.0,130591.0,248406.0,...,230540.0,196352.0,256790.0,167383.0,183772.0,311804.0,45899.0,149461.0,216344.0,200614.0
2016,169570.0,224860.0,523530.0,48684.0,70200.0,168983.0,140424.0,166580.0,133274.0,249656.0,...,241905.0,212621.0,291757.0,170594.0,194372.0,318446.0,51381.0,154381.0,218323.0,199671.0
2017,159106.0,236392.0,495785.0,41936.0,67322.0,135939.0,142747.0,166299.0,133718.0,236255.0,...,252793.0,214083.0,170514.0,154487.0,197919.0,326822.0,54352.0,154066.0,196907.0,123052.0
2018,159745.0,245560.0,540469.0,42700.0,68107.0,124522.0,150290.0,173259.0,136650.0,256218.0,...,262089.0,233318.0,195448.0,161754.0,203572.0,342635.0,60984.0,154941.0,200074.0,136241.0
2019,179777.0,312564.0,586674.0,43594.0,80055.0,145586.0,160449.0,179644.0,141226.0,274450.0,...,272585.0,253901.0,245917.0,163577.0,210326.0,355738.0,66807.0,160780.0,219316.0,164748.0
2020,186358.0,146386.0,285139.0,128964.0,194376.0,138603.0,144536.0,249207.0,213653.0,285450.0,...,162925.0,243135.0,181226.0,277533.0,239587.0,198307.0,223164.0,158472.0,365896.0,195813.0
2021,303561.0,137283.0,204643.0,142633.0,150689.0,257485.0,228622.0,338967.0,70215.0,223069.0,...,209059.0,285327.0,251230.0,206496.0,236271.0,165170.0,382089.0,207685.0,326003.0,347144.0


In [127]:
states_list[0]

'ANDAMAN AND NICOBAR'

In [128]:
a[0]

'Andaman'

In [140]:
a1=a.copy()

In [141]:
a1

['Andaman',
 'Dadra_And_Nagar_Haveli',
 'Daman_&_Diu',
 'Lakshadweep',
 'Nagaland',
 'Andhra_Pradesh',
 'Arunachal_Pradesh',
 'Assam',
 'Bihar',
 'Chhattisgarh',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal_Pradesh',
 'Jammu_And_Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Madhya_Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Odisha',
 'Puducherry',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil_Nadu',
 'Tripura',
 'Uttar_Pradesh',
 'Uttarakhand',
 'West_Bengal',
 'All Districts']

In [149]:
a[0]

'Andaman'

In [178]:
for i in range(1,11):
        n=li[i-1]
        n.fillna("", inplace = True) 
        if(i==1):

            n=n[["State Name","District Name","Total households applied for job card"]]
            n.rename(columns={"Total households applied for job card":yr[i-1]},inplace=True)
            final=n
        else:
            #n.drop(columns={"Unnamed: 0","Sno."},inplace=True)
            n=n[["State Name","District Name","Total households applied for job card"]]
            n.rename(columns={"Total households applied for job card":yr[i-1]},inplace=True)
            n.drop(columns={'State Name','District Name'},inplace=True)
            final=pd.concat([final,n],axis=1,sort=False)
            

In [167]:
final

,State Name,District Name,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ANDAMAN AND NICOBAR,NICOBARS,2882,5149,5512,4245,5737,5620,5585,0,74746.0,74780.0
1,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,17687,18292,19765,20511,20967,20234,19848,0,23827.0,23848.0
2,ANDAMAN AND NICOBAR,SOUTH ANDAMAN,24827,25048,25374,25278,25337,10170,10081,0,53981.0,53789.0
3,DADRA & NAGAR HAVELI,DADRA AND NAGAR HAVELI,7845,8176,8157,8157,8157,8157,9141,0,21738.0,21738.0
4,DAMAN & DIU,DAMAN,0,0,0,0,0,0,0,0,35754.0,35744.0
...,...,...,...,...,...,...,...,...,...,...,...,...
633,WEST BENGAL,PASCHIM MEDINIPUR,874923,898113,921649,944010,964128,976011,726728,736253,NaN,NaN
634,WEST BENGAL,PURBA MEDINIPUR,670770,685931,707296,727831,742591,758838,773259,782936,NaN,NaN
635,WEST BENGAL,PURULIA,446947,456861,469053,479954,488525,497750,499683,505141,NaN,NaN
636,WEST BENGAL,SILIGURI MAHAKUMA PARISAD,86873,91666,96178,99874,101770,104666,96516,97452,NaN,NaN


In [179]:
final.loc[637,"State Name"]="Total"
final.loc[637,"District Name"]=0

In [180]:

final=final.replace(np.nan,"",regex=True)
final

,State Name,District Name,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,ANDAMAN AND NICOBAR,NICOBARS,2882,5149,5512,4245,5737,5620,5585,0,74746,74780
1,ANDAMAN AND NICOBAR,NORTH AND MIDDLE ANDAMAN,17687,18292,19765,20511,20967,20234,19848,0,23827,23848
2,ANDAMAN AND NICOBAR,SOUTH ANDAMAN,24827,25048,25374,25278,25337,10170,10081,0,53981,53789
3,DADRA & NAGAR HAVELI,DADRA AND NAGAR HAVELI,7845,8176,8157,8157,8157,8157,9141,0,21738,21738
4,DAMAN & DIU,DAMAN,0,0,0,0,0,0,0,0,35754,35744
...,...,...,...,...,...,...,...,...,...,...,...,...
633,WEST BENGAL,PASCHIM MEDINIPUR,874923,898113,921649,944010,964128,976011,726728,736253,,
634,WEST BENGAL,PURBA MEDINIPUR,670770,685931,707296,727831,742591,758838,773259,782936,,
635,WEST BENGAL,PURULIA,446947,456861,469053,479954,488525,497750,499683,505141,,
636,WEST BENGAL,SILIGURI MAHAKUMA PARISAD,86873,91666,96178,99874,101770,104666,96516,97452,,
